## Using Trained Model

In [11]:
import os
import cv2
import shutil
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions


In [12]:
def get_model():
  # load model
  model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
  # set trainable params as false.
  model.trainable = False

  # add new classifier layers
  flat1 = Flatten()(model.layers[-1].output)
  class1 = Dense(1024, activation='relu')(flat1)
  class2 = Dense(512, activation='relu')(class1)
  class3 = Dense(256, activation='relu')(class2)
  class4 = Dense(64, activation='relu')(class3)
  output = Dense(16, activation='softmax')(class4)

  # define new model
  model = Model(inputs=model.inputs, outputs=output)

  return model


In [34]:
def get_label(path,image_name,model):
  input_image = path+image_name
  image_size = (224,224)
  class_subset=['Switch','ToggleButton','ImageButton','ProgressBarHorizontal','SeekBar','RadioButton', 'CheckedTextView', 'Button', 'NumberPicker',
                'EditText','ImageView', 'CheckBox', 'ProgressBarVertical', 'TextView', 'RatingBar','Spinner']
  # Create Model.
  #model=get_model()
  # Load trained weights.
  #model.load_weights("./checkpoints")
  #model.load_weights("/content/drive/MyDrive/Colab Notebooks/795/Model_Backup/checkpoint").expect_partial()
  image = load_img(input_image, target_size=(224, 224))
  # convert the image pixels to a numpy array
  image = img_to_array(image)
  # reshape data for the model
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  # prepare the image for the VGG model
  image = preprocess_input(image)
  # predict the probability across all output classes
  yhat = model.predict(image)
  yhat=yhat.reshape(16)
  label = np.argmax(yhat)
  label=class_subset[label]
  return label

## Generating XML File.

In [35]:
% mkdir xml

In [37]:
import xml.etree.ElementTree as ET
import json

# destination path for xml files
xml_files = "/content/xml"
# path where the output of UIED model is been saved for github it's 
# big_folder="UIED/data/output/ReDrawModel/"
big_folder = "/content/drive/MyDrive/Colab Notebooks/795/ReDrawModel"
l=os.listdir(big_folder)
model = get_model()
# Path to saved weights.
model.load_weights("/content/drive/MyDrive/Colab Notebooks/795/Model_Backup/checkpoint").expect_partial()

for img_folder in l:
      print("processing:",img_folder)
      path=big_folder+"/"+img_folder+"/"
      file_p=path+img_folder+".json"
      f = open(file_p)
      json_dict = json.load(f)
      data = ET.Element('?xml')
      data.set('version','1.0')
      data.set('encoding','UTF-8')
      data.set('standalone','yes')

      element1 = ET.SubElement(data, 'hierarchy')
      element1.set('rotation', '0')
      for i in json_dict:
            widget='android.widget.'+str(get_label(path,i,model))
            #if count==0:
            s_elem1 = ET.SubElement(element1, 'node')
            s_elem1.set('index', i)
            s_elem1.set('text', '')
            s_elem1.set('resource-id', '')
            s_elem1.set('class', widget)
            s_elem1.set('package', 'com.pandora.android')
            s_elem1.set('content-desc', '')
            s_elem1.set('checkable', 'false')
            s_elem1.set('checked', 'false')
            s_elem1.set('clickable', 'false')
            s_elem1.set('enabled', 'false')
            s_elem1.set('focusable', 'false')
            s_elem1.set('focused', 'false')
            s_elem1.set('scrollable', 'false')
            s_elem1.set('long-clickable', 'false')
            s_elem1.set('password', 'false')
            s_elem1.set('selected', 'false')
            s_elem1.set('bounds', json_dict[i])
            f.close()
      b_xml = ET.tostring(data)
      fname= xml_files+"/"+img_folder+".xml"
      with open(fname, "wb") as f:
        f.write(b_xml)


processing: meet_2.jpg
